# Validation of CCIC against HAIC-HIWC IKP-2 measurements

This notebook compiles the code required to collocate and analyse the IKP-2 measurements during the HAIC-HIWC campaign.

Files containing the IKP-2 measurements are prepared from the raw format given by the data provider into a suitable format elsewhere.

The notebook uses `if False:` statements to comment out code needs to be run only once.

In [ ]:
# Import libraries

import datetime
import glob
import os
from pathlib import Path
import pickle
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import tqdm
import xarray as xr

from ccic.data.gridsat import GridSat
from ccic.data.cpcir import CPCIR
from ccic.plotting import set_style
from ccic.validation import get_latlon_bins, resample_data

# Set plotting style
set_style()

# Set Cartopy directory
os.environ["CARTOPY_USER_BACKGROUNDS"] = '/mnt/data_sun/ccic/misc'
Image.MAX_IMAGE_PIXELS = None

In [ ]:
# Links to the folders containing the IKP-2 data files
orders = [
    'amell60263', # 2014, https://doi.org/10.5065/D6WW7GDS
    'amell60877', # 2015, https://doi.org/10.5065/D61N7ZV7
    'amell61210', # 2015, https://doi.org/10.5065/D6RN36KJ
    'amell32226'  # 2018, https://doi.org/10.26023/8V5Y-GB2E-CX07
]

orders_path = Path('/mnt/data_sun/ccic/flight_campaigns/HAICHIWC/orders/friendly')

In [ ]:
# Compile the spatial and temporal boundaries of each flight
# Also keep the paths of the flights
flight_tracks = []
timestamps_flight_boundaries = []
roi_flight_boundaries = []
roi_campaign = [] # Record by campaign as well
for i, order in enumerate(orders):
    files_order = sorted(glob.glob(str(orders_path / order / '*csv')))
    flight_tracks_order = []
    _roi_campaign = []
    for f in files_order:
        df = pd.read_csv(f)
        # Filter only measurements where TWC has a valid (non-negative) value
        df_filter = df[(pd.isna(df.TWC_gm3) == False) & (df.TWC_gm3 >= 0)].reset_index(drop=True)
        # Format time
        df_filter['UTC'] = pd.to_datetime(df_filter.UTC)
        # Record the the flight tracks and boundaries
        flight_tracks_order.append(df_filter[['longitude', 'latitude']].values)
        timestamps_flight_boundaries.append([df_filter.UTC.min(), df_filter.UTC.max()])
        roi_flight_boundaries.append([df_filter.longitude.min(), df_filter.latitude.min(), df_filter.longitude.max(), df_filter.latitude.max()])
        _roi_campaign.append(roi_flight_boundaries[-1])
    roi_campaign.append(_roi_campaign)
    flight_tracks.append(flight_tracks_order)

# Having a ROI for each campaign will be more convenient later on
roi_campaign_flight = []
for c in roi_campaign:
    for _ in c:
        roi_campaign_flight.append(np.array(c).min(axis=0)[:2].tolist() + np.array(c).max(axis=0)[2:].tolist())

In [ ]:
# Sanity check: plot the flight tracks

# Create a figure and axis with the Robinson projection
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

# Set the extent to global
ax.set_global()

ax.add_feature(cfeature.COASTLINE)

for campaign_i in range(4):
    for track in flight_tracks[campaign_i]:
        ax.scatter(track[:,0], track[:,1], c=f'C{campaign_i}', s=1, transform=ccrs.PlateCarree())

plt.show()

In [ ]:
# Make a list with the file names to download
files_gridsat_to_download = []
files_cpcir_to_download = []
for (t_start, t_end)  in timestamps_flight_boundaries:
    for t in pd.date_range(t_start.floor('H'), t_end.ceil('H'), freq='H', inclusive='both'):
        files_gridsat_to_download.append(t.strftime('GRIDSAT-B1.%Y.%m.%d.%H.v02r01.nc'))
        files_cpcir_to_download.append(t.strftime('merg_%Y%m%d%H_4km-pixel.nc4'))

# Gridsat files only exist every three hours, therefore clear list from files that will not exist
files_gridsat_to_download = set.intersection(
    set(files_gridsat_to_download),
    set([t.strftime('GRIDSAT-B1.%Y.%m.%d.%H.v02r01.nc') for t in pd.date_range(pd.Timestamp('2014'), pd.Timestamp('2019'), freq='3H', inclusive='both')])
)
files_cpcir_to_download = set(files_cpcir_to_download)

In [ ]:
# Download the data
INPUT_DATA_PATH = Path('/data/ccic_input_haichiwc_ikp2')

if False:
    for f in tqdm.tqdm(files_gridsat_to_download, ncols=80):
        GridSat.download(f, str(INPUT_DATA_PATH / 'gridsat' / f))

    for f in tqdm.tqdm(files_cpcir_to_download, ncols=80):
        CPCIR.download(f, str(INPUT_DATA_PATH / 'cpcir' / f))

Now we define the commands to run CCIC retrievals.

We need retrievals $\pm 15$ minutes from the IKP-2 measurement, if the product allows for it. In addition, we'll only do inference for the ROI covered by the flight track. We'll use a generous ROI to stay away from any potential edge effect.

Note: at the time of writing this notebook, the `ccic process` command accepts the following arguments:

```{toggle}
$ ccic process --help
usage: ccic process [-h] [--input_path path] [--targets target [target ...]] [--tile_size N] [--overlap N] [--device dev] [--precision 16/32]
                    [--output_format netcdf/zarr] [--database_path path] [--roi x x x x] [--n_processes n] [--inpainted_mask]
                    [--confidence_interval CONFIDENCE_INTERVAL]
                    model cpcir/gridsat output t1 [t2]

Run CCIC retrieval.

positional arguments:
  model                 Path to the trained CCIC model.
  cpcir/gridsat         For which type of input to run the retrieval.
  output                Folder to which to write the output.
  t1                    The time for which to run the retrieval.
  t2                    If given, the retrieval will be run for all files in the time range t1 <= t <= t2

options:
  -h, --help            show this help message and exit
  --input_path path     Path to a local directory containing input files. If not given, input files will be downloaded using pansat.
  --targets target [target ...]
  --tile_size N         Tile size to use for processing.
  --overlap N           Tile size to use for processing.
  --device dev          The name of the torch device to use for processing.
  --precision 16/32     The precision with which to run the retrieval. Only has an effect for GPU processing.
  --output_format netcdf/zarr
                        The output format in which to store the output: 'zarr' or 'netcdf'. 'zarr' format applies a custom filter to allow storing 'tiwp' fields as
                        8-bit integer, which significantly reduces the size of the output files.
  --database_path path  Path to the database to use to log processing progress.
  --roi x x x x
  --n_processes n
  --inpainted_mask      Create a variable `inpainted` indicating if the retrieved pixel is inpainted (the input data was NaN).
  --confidence_interval CONFIDENCE_INTERVAL
                        Width of the equal-tailed confidence interval to use to report retrieval uncertainty of scalar retrieval targets. Must be within [0, 1].
```

We'll use a GPU from the cluster

In [ ]:
# Some constants matching how the container will be executed
model = '/data/ccic.pckl'       # CCIC model to use
input_path = Path('/data/input')
output_path = Path('/data/output')   # Where to store the output
delta_roi = 2.5                 # Expand the region covered by the flight track with this many degrees
commands = []                   # List to store shell commands to run the retrieval

# Build the commands
# We will use the ROI for each campaign rather than the ROI for each flight,
# which avoids problem later when creating the collocations
for ((time_start, time_end), roi) in zip(timestamps_flight_boundaries, roi_campaign_flight):
    # Add margins to ROI
    roi[0] = roi[0] - delta_roi
    roi[1] = roi[1] - delta_roi
    roi[2] = roi[2] + delta_roi
    roi[3] = roi[3] + delta_roi

    roi_str = ' '.join([str(e) for e in roi])
    
    # We'll use the .floor and .ceil methods of pd.Timestamp for the +/- 15 min requirement
    args_cpcir = [model, 'cpcir', output_path / 'cpcir',
                  time_start.floor('H').strftime('%Y-%m-%dT%H:%M'),
                  time_end.ceil('H').strftime('%Y-%m-%dT%H:%M'),
                  roi_str,
                  input_path / 'cpcir']
    args_gridsat = [model, 'gridsat', output_path / 'gridsat',
                    time_start.floor('3H').strftime('%Y-%m-%dT%H:%M'), # 3-hour resolution
                    time_end.ceil('3H').strftime('%Y-%m-%dT%H:%M'), # 3-hour resolution
                    roi_str,
                    input_path / 'gridsat']
    commands.append('ccic process {:} {:} {:} {:} {:} --roi {:} --input_path {:} --targets tiwc --device cuda'.format(*args_cpcir))
    commands.append('ccic process {:} {:} {:} {:} {:} --roi {:} --input_path {:} --targets tiwc --device cuda'.format(*args_gridsat))

# Save these commands into a shell script and execute in the cluster for speed
# The shell script is adapted for the cluster outside of this notebook
if False:
    with open('validation_haic_hiwc_ikp2.sh', 'w') as handle:
        handle.write('\n'.join(commands))

We'll do a sanity check: the retrievals produced cover the flight tracks in space and time

In [ ]:
def filename_to_ts(f):
    """Extract the timestamp from the filenames"""
    return datetime.datetime.strptime(Path(f).name.split('_')[-1], '%Y%m%d%H00.nc')

output_files = {
    'cpcir': {filename_to_ts(f): f for f in sorted(glob.glob('/data/ccic_output_haichiwc_ikp2/cpcir/*nc'))},
    'gridsat': {filename_to_ts(f): f for f in sorted(glob.glob('/data/ccic_output_haichiwc_ikp2/gridsat/*nc'))}
}

# Re-structure the dict and apply time formatting
for product, file_dict in output_files.items():
    output_files[product] = [
        np.array(list(file_dict.keys()), dtype='datetime64'),
        np.array(list(file_dict.values()))
    ]

for i, ((t_start, t_end), (lon_min, lat_min, lon_max, lat_max)) in tqdm.tqdm(enumerate(zip(timestamps_flight_boundaries, roi_flight_boundaries)), total=len(timestamps_flight_boundaries)):
    product = 'cpcir'
    unit = '1H'
    times_ir, files_ir = output_files[product]
    idx_times = (t_start.floor(unit) <= times_ir) & (times_ir <= t_end.ceil(unit))
    assert idx_times.any()
    retrievals_flight = files_ir[idx_times]
    lon_min_ds, lat_min_ds, lon_max_ds, lat_max_ds = [], [], [], []
    for f in retrievals_flight:
        ds = xr.open_dataset(f)
        lon_min_ds.append(ds.longitude.values.min())
        lat_min_ds.append(ds.latitude.values.min())
        lon_max_ds.append(ds.longitude.values.max())
        lat_max_ds.append(ds.latitude.values.max())
        assert min(lon_min_ds) <= lon_min <= max(lon_max_ds)
        assert min(lat_min_ds) <= lat_min <= max(lat_max_ds)

        # This is to check that the ROIs for the retrieval are all the same
        assert np.unique(lon_min_ds).size == 1
        assert np.unique(lat_min_ds).size == 1
        assert np.unique(lon_max_ds).size == 1
        assert np.unique(lat_max_ds).size == 1

No AssertionError was raised, which means that all flights have retrievals. Next, we'll resample the flight data

In [ ]:
def prepare_csv(f):
    # Read CSV
    df = pd.read_csv(f, parse_dates=['UTC'])[['UTC', 'latitude', 'longitude', 'altitude_pressure_metres', 'TWC_gm3']]
    # Keep only valid TWC measurements
    df = df[(pd.isna(df.TWC_gm3) == False) & (df.TWC_gm3 >= 0)].reset_index(drop=True)
    # Rename
    df = df.rename(columns={'UTC': 'time', 'altitude_pressure_metres': 'altitude', 'TWC_gm3': 'twc'})
    # Set time as index
    df = df.set_index('time')
    return xr.Dataset(df)

RETRIEVALS_PATH = Path('/data/ccic_output_haichiwc_ikp2')
RESAMPLE_OUTPUT_PATH = Path('/data/ccic_haichiwc_ikp2_resampled')

if False:
    flight_count = 0
    for order in orders:
        files_order = sorted(glob.glob(str(orders_path / order / '*csv')))
        for f in files_order:
            ds = prepare_csv(f)
            ds_t_start = pd.to_datetime(ds.time.values[0])
            for product, freq in {'cpcir': '1H', 'gridsat': '3H'}.items():
                # Open a retrieval sample just to get the grid
                retrieval_sample = f"ccic_{product}_{ds_t_start.ceil(freq).strftime('%Y%m%d%H')}00.nc"
                bin_lats, bin_lons = get_latlon_bins(RETRIEVALS_PATH / product / retrieval_sample)
                # Resample
                with warnings.catch_warnings():
                    warnings.filterwarnings('ignore', 'Index.ravel returning ndarray')
                    # Note: the current version of resample_data can produce files that do not have
                    #       any variable. Example: measurements start at T = 20:59. The 20 h file
                    #       will be produced with no variables, but there are no collocations with it
                    resample_data(ds, ['twc'], bin_lons, bin_lats, RESAMPLE_OUTPUT_PATH / product / order,
                                f"in_situ_{product}_{{year:04}}{{month:02}}{{day:02}}{{hour:02}}.nc")
            flight_count += 1
            print(f"Processed {flight_count} flights", end='\r')

Collocate the data:

In [ ]:
RETRIEVALS_IKP2_PATH = Path('/data/ccic_haichiwc_ikp2_resampled_and_collocated')

if False:
    with tqdm.tqdm(total=len(list((RESAMPLE_OUTPUT_PATH).glob('**/*.nc')))) as pbar:
        for order in orders:
            for product, freq in {'cpcir': 1, 'gridsat': 3}.items():
                resampled_files = list((RESAMPLE_OUTPUT_PATH / product / order).glob('*.nc'))
                for f in resampled_files:
                    pbar.update()
                    ts = pd.to_datetime(datetime.datetime.strptime(f.name.split('_')[-1], '%Y%m%d%H.nc'))
                    if (ts.hour % freq) != 0:
                        # Deal with the 3-hour resolution of GridSat
                        continue
                    ds_resampled = xr.open_dataset(f)
                    retrieval_base_fname = f'ccic_{product}_{ts.strftime("%Y%m%d%H")}'
                    ds_retrieval = xr.open_dataset(RETRIEVALS_PATH / product / f'{retrieval_base_fname}00.nc')

                    # Sanity checks:
                    assert (abs(ds_resampled.latitude - ds_retrieval.latitude) <= 0.001).all()
                    assert (abs(ds_resampled.longitude - ds_retrieval.longitude) <= 0.001).all()
                    assert np.allclose(ds_resampled.altitude * 1e3, ds_retrieval.altitude)

                    # Add in-situ tiwp to retrieval
                    if 'twc' not in ds_resampled:
                        # This is possible, for an explanation see the previous code cell
                        continue
                    # Filter by time: will only have an effect for GridSat
                    #                 also need to handle floating point issues with datetime64...
                    idx_time = ds_resampled.time.values.astype('datetime64[m]') == ds_retrieval.time.values.astype('datetime64[m]')
                    ds_resampled = ds_resampled.sel(time=idx_time)
                    
                    # Sanity checks
                    assert (abs(ds_resampled.longitude.values - ds_retrieval.longitude.values) < 0.001).all()
                    assert (abs(ds_resampled.latitude.values - ds_retrieval.latitude.values) < 0.001).all()
                    
                    dims = list(ds_retrieval.dims)
                    ds_retrieval["twc_ikp2"] = (dims, ds_resampled.transpose(*dims).twc.data)

                    # Write to disk
                    ds_retrieval.to_netcdf(RETRIEVALS_IKP2_PATH / product / order / f'{retrieval_base_fname}_ikp2.nc')

Finally, we construct arrays containing all valid retrieval-measurement collocations and locations, sorted by campaign and product

In [ ]:
if False:
    files_processed = 0
    data = {order: {'cpcir': None, 'gridsat': None} for order in orders}
    for order, collocations in data.items():
        for product in collocations.keys():
            files = Path(RETRIEVALS_IKP2_PATH / product / order).glob('*nc')
            data_campaign = []
            for f in files:
                ds = xr.open_dataset(f).transpose('longitude', 'latitude', 'altitude', 'time')
                valid = np.isfinite(ds.tiwc) * (ds.tiwc >= 0) * np.isfinite(ds.twc_ikp2) * (ds.twc_ikp2 >= 0)
                valid_idx = np.nonzero(valid.values)
                longitudes = ds.longitude.values[valid_idx[0]]
                latitudes = ds.latitude.values[valid_idx[1]]
                tiwc = ds.tiwc.values[valid]
                twc_ikp2 = ds.twc_ikp2.values[valid]
                data_campaign.append(np.array([longitudes, latitudes, tiwc, twc_ikp2]).T)
                files_processed += 1
                print(f'Files processed: {files_processed}', end='\r')
            pd.DataFrame(np.concatenate(data_campaign, axis=0), columns=['longitude', 'latitude', 'tiwc', 'twc_ikp2']).to_xarray()
            data[order][product] = pd.DataFrame(np.concatenate(data_campaign, axis=0), columns=['longitude', 'latitude', 'tiwc', 'twc_ikp2']).to_xarray()
    with open('/mnt/data_sun/ccic/analyses/haic-hiwc/collocated.pickle', 'wb') as handle:
        pickle.dump(data, handle)

Create the plots

In [ ]:
# First load the pre-computed data
with open('/mnt/data_sun/ccic/analyses/haic-hiwc/collocated.pickle', 'rb') as handle:
    data = pickle.load(handle)

# Re-organize dictionary keys
orders_map = {
    'amell60263': 0, # 2014, https://doi.org/10.5065/D6WW7GDS
    'amell60877': 1, # 2015, https://doi.org/10.5065/D61N7ZV7
    'amell61210': 2, # 2015, https://doi.org/10.5065/D6RN36KJ
    'amell32226': 3  # 2018, https://doi.org/10.26023/8V5Y-GB2E-CX07
}

data = {v: data[k] for k, v in orders_map.items()}

In [ ]:
fig = plt.figure(frameon=False)

axs = [
    fig.add_axes([0, 0, 1, 1], projection=ccrs.PlateCarree()),
    fig.add_axes([0.33, 0.175, 0.35, 0.4], projection=ccrs.PlateCarree())
]

colors = {0: 'C1', 1: 'C0', 2: 'C3', 3: 'w'}
labels = {
    0: 'Jan--Feb 2014', 1: 'May 2015', 2: 'Aug 2015', 3: 'Jul--Aug 2018'
}


for campaign_idx, campaign_data in data.items():
    if campaign_idx == 0:
        ax = axs[1]
    else:
        ax = axs[0]

    for ds in campaign_data.values():
        ax.scatter(ds.longitude, ds.latitude, s=.5, c=colors[campaign_idx], transform=ccrs.PlateCarree(), rasterized=True)

for ax in axs:
    ax.background_img(name='blue_marble_dec', resolution='low')

axs[0].gridlines(crs=ccrs.PlateCarree(), draw_labels={"top": "x", "left": "y"}, linestyle='dotted', alpha=0.3)
axs[1].gridlines(crs=ccrs.PlateCarree(), draw_labels={"bottom": "x", "right": "y"}, ylabel_style={'color': 'white'}, linestyle='dotted', alpha=0.3)

axs[0].set_extent([-160, -45, 0, 40], crs=ccrs.PlateCarree())
axs[1].set_extent([120, 145, -20, -7.5], crs=ccrs.PlateCarree())

legend_elements = [matplotlib.lines.Line2D([0], [0], lw=0, color=colors[i], label=labels[i], markerfacecolor=colors[i], marker='s') for i in range(4)]

axs[0].legend(handles=legend_elements, loc='upper left', facecolor='lightgray', handlelength=0.5)

fig.savefig('haic_hiwc_coverage.pdf', bbox_inches='tight')

plt.close(fig)

In [ ]:
bins = [0] + np.logspace(-4, 1, 55, endpoint=False).tolist() + [10]

norm = matplotlib.colors.LogNorm(1e-2, 1e2)
txtcol = 'C0'
diagcol = 'orangered'

scale_figsize = 1.25
fig, axs = plt.subplots(ncols=2, nrows=4, figsize=(4*scale_figsize,8*scale_figsize), sharex=True, sharey=True)

label_i = 0
for campaign_idx, campaign_data in data.items():
    for product_idx, product in enumerate(['CPCIR', 'GridSat']):
        data_plot = campaign_data[product.lower()]
        ax = axs[campaign_idx,product_idx]

        # Keep only valid values
        tiwc = data_plot.tiwc.values
        twc = data_plot.twc_ikp2.values
        idxs = np.isfinite(twc) & np.isfinite(tiwc) # This should be redundant with a pre-processing step
        twc = twc[idxs]
        tiwc = tiwc[idxs]

        H, _, _ = np.histogram2d(twc, tiwc, bins=bins, density=True)

        normalization = np.tile(np.sum(H * np.diff(bins)[None], axis=1, keepdims=True), (1, H.shape[1]))
        H = np.divide(H, normalization, out=np.full_like(H, np.nan), where=normalization > 0)

        mesh = ax.pcolormesh(bins[1:-1], bins[1:-1], H[1:-1, 1:-1].T, norm=norm, rasterized=True)

        corr = np.corrcoef(twc, tiwc)[0, 1]
        bias = (tiwc - twc).mean() / twc.mean()
        props = dict(facecolor='white', alpha=1.0, edgecolor="k")
        # ax.text(1e-1, 1.5e-3, f"Corr.: {corr:0.2f} \n Bias: {100 * bias:0.2f}\%",
        #         fontsize=12, color=txtcol, ha="left", va="bottom", bbox=props)
        stats_pos_x = 2e-1
        stats_string = f"{corr:0.2f}\n{100 * bias:0.2f}\%"
        ax.text(stats_pos_x, 3e-4, stats_string,
                fontsize=10, color=txtcol, ha="left", va="bottom", bbox=props)
        # ax.set_title(f'({chr(97 + (i + i_campaign-1))}) {product}', fontsize=14)
        # pos_text_labels = (2.5e-1, 1.5e-3)
        pos_text_labels = (1.25e-4, 2.5)
        ax.text(*pos_text_labels, f'({chr(97 + label_i)})', fontsize=14)
        label_i += 1


for ax in axs.ravel():
    ax.set_xlim(1e-4, 1e1)
    ax.set_ylim(1e-4, 1e1)
    ax.plot(bins, bins, c="grey", ls="--")
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_aspect('equal')

# for ax in axs[:,0]:
#     ax.set_ylabel(r'CCIC TIWC [$\si{\gram\per\cubic\meter}$]')
axs[1,0].set_ylabel(r'CCIC TIWC [$\si{\gram\per\cubic\meter}$]')
# for ax in axs[-1,:]:
#     ax.set_xlabel(r'HAIC-HIWC TWC [$\si{\gram\per\cubic\meter}$]')
fig.text(0.5, 0.06, r'HAIC-HIWC TWC [$\si{\gram\per\cubic\meter}$]', ha='center')

for i, ax in enumerate(axs[:,-1]):
    ax.yaxis.set_label_position("right")
    ax.set_ylabel(labels[i], rotation=-90, labelpad=15)

axs[0,0].set_title('CPCIR', loc='center')
axs[0,1].set_title('GridSat', loc='center')

#     fig.colorbar(mesh, cax=axs[2], label=r'p(IWC $|$ TWC) [$\si{\per\gram\cubic\meter}$]', extend='both')

ax_cbar = fig.add_axes([0,0,1,0.05])
ax_cbar.set_axis_off()
cbar = fig.colorbar(mesh, ax=ax_cbar, label=r'p(TIWC $|$ TWC) [$\si{\per\gram\cubic\meter}$]', extend='both', orientation='horizontal', fraction=0.4)
cbar.ax.xaxis.set_label_position('top')

fig.subplots_adjust(hspace=0.075, wspace=0.025)

fig.savefig(f'haic-hiwc_tiwc_vs_twc_campaigns_individual.pdf', bbox_inches='tight')

plt.show()